# genetic algorithm framework
## fill in the TODOs

In [16]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple

Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


def generate_genome(length: int) -> Genome: 
    # TODO: generate a random list of 0s and 1s of 'length' 
    # hint: randint(a,b) will return a random integer between a, b including the endpoints
    genome = []
    for i in range(length):
        genome.append(randint(0, 1))
    return genome

def generate_population(size: int, genome_length: int) -> Population: 
    # TODO: return a list of Genomes of 'genome_length' where each genome has len = 'size'
    population=[]
    for i in range(size):
        population.append(generate_genome(genome_length))
    return population



def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    # TODO:return offsprings for a single point cross over
    temp=randint(0,len(a)-1)
    offspring1  = a[:temp] + b[temp:]
    offspring2 =  b[:temp] + a[temp:]
    return offspring1, offspring2


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # TODO: mutate 'genome' at 'num' spots, each with 'probability'
    
    mutated_genome = genome
    temp=random()
    for i in range(num):
        if temp <=probability:
            mutated_genome[randint(0,len(genome)-1)]=randint(0,1)
    return mutated_genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int: # returns the sum of the fitness of the population
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population: # selects 2 'parents' from the population
    # which are sampled with proabability proportional to their fitness value
    
    return sample( # sample function: given a list 'population' samples 'k' uniform random items
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )


def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    # TODO: given a fitness function and population, make a list that when sampled from uniformly 
    # returns a particular genome with probability proportional to its fitness value.
    # hint: If this is not intuitive, delete the body of selection_pair, and try to rewrite it with another approach (you may not need this function
    # in that case)
    # result = []
    # for i in range(2):
    #     temp=randint(1,population_fitness)
    #     last=0
    #     sum=0
    #     for j in range (len(population)-1):
    #         last=sum
    #         sum+=fitness_func(population(j))
    #         if temp<=sum and temp>last:
    #             temp=j
    #     result.append(population(temp))

    # return result

    result =[]
    for i in range(len(population)):
        result+=[population[i]]*fitness_func(population[i])


    return result

def sort_population(population: Population, fitness_func: FitnessFunc) -> Population: # sorts the population in descending order of fitness value
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str: # represents the genome list as a string
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            print("BREAKING")
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2)-1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i


### write a fitness function that maximizes the number of zeros in a genome <br>
What will the optimal solution be? ______________

In [23]:
def fitness_zeros(genome: Genome) -> int:
    # TODO: write a fitness function for the max number of zeros problem
    score=0
    for i in range(len(genome)):
        if genome[i]==0:
            score+=1
    return score
   


def generate_custom_population():
    return generate_population(size = 10,genome_length = 10) # input the length of the genome and size of the population
run_evolution(generate_custom_population, fitness_zeros, 10, selection_pair, single_point_crossover, mutation, 100, print_stats)


GENERATION 00
Population: [0010011001, 0011010001, 1000000111, 1001110010, 0100110011, 1011011011, 1111011100, 1011111100, 0110111110, 1111110101]
Avg. Fitness: 4.200000
Best: 0010011001 (6.000000)
Worst: 1111110101 (2.000000)

GENERATION 01
Population: [1000000001, 0010011001, 0011010001, 1000000111, 0100110010, 0010011011, 0010011111, 0011111101, 1011011011, 1011111100]
Avg. Fitness: 5.000000
Best: 1000000001 (8.000000)
Worst: 1011111100 (3.000000)

GENERATION 02
Population: [1000000001, 1000000001, 0010000001, 0000011001, 0010011001, 0010011001, 0010011001, 0010110010, 1010011001, 0011011011]
Avg. Fitness: 6.400000
Best: 1000000001 (8.000000)
Worst: 0011011011 (4.000000)

GENERATION 03
Population: [1000000001, 1000000001, 1000000001, 0000010001, 0110000001, 0110000001, 0010001001, 0010011001, 0010011001, 0010111001]
Avg. Fitness: 7.000000
Best: 1000000001 (8.000000)
Worst: 0010111001 (5.000000)

GENERATION 04
Population: [1000000001, 1000000001, 0000001001, 0000010001, 1000000001, 0

([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]],
 12)

## write a fitness function for the knapsack problem:
An example: given 4 items, <br> <br>
$[apple, weight = 10, value = 10]$ <br>
$[laptop, weight = 70, value = 100]$ <br>
$[candle, weight = 3, value = 1]$ <br>
$[money, weight = 2, value = 200]$ <br>
<br>
find the items you will choose if you have a bag of weight 80 that maximise the value

In [28]:
# the list_of_items will be in the format [[size, value], [size, value],.....] an example is given below
list_of_items_eg = [[10,10], [10,2], [4,2], [100, 6], [9, 100], [45,52], [9, 10], [5,6], [12,31], [12, 13]] 
bag_limit_eg = 100
def fitness_knapsack(genome:Genome, list_of_items: List[List[int]], bag_limit: int) -> int:
    # TODO write a fitness function for the knapsack problem
    # hint: think about what improves the 'value' of the knapsack
    score=0
    weight=0
    for i in range(len(genome)):
        if genome[i]==1:
            score+=list_of_items[i][1]
            weight+=list_of_items[i][0]
    if weight>bag_limit:
        score=0
    return score

        
run_evolution(generate_custom_population, lambda genome: fitness_knapsack(genome, list_of_items_eg, 100), 100, selection_pair, single_point_crossover, mutation, 100, print_stats)


GENERATION 00
Population: [1000111110, 1000000111, 0010001001, 0010000101, 0110001000, 1111011011, 1111011001, 1001110000, 1101101110, 1111000001]
Avg. Fitness: 32.900000
Best: 1000111110 (209.000000)
Worst: 1111000001 (0.000000)

BREAKING


([[1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
  [1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
  [0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
  [0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
  [0, 1, 1, 0, 0, 0, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
  [1, 1, 1, 1, 0, 1, 1, 0, 0, 1],
  [1, 0, 0, 1, 1, 1, 0, 0, 0, 0],
  [1, 1, 0, 1, 1, 0, 1, 1, 1, 0],
  [1, 1, 1, 1, 0, 0, 0, 0, 0, 1]],
 0)